In [ ]:
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.11.0+cu113.html

Looking in links: https://data.pyg.org/whl/torch-1.11.0+cu113.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.9 MB/s eta 0:00:00
  Created wheel for torch-scatter: filename=torch_scatter-2.1.2-cp310-cp310-linux_x86_64.whl size=507268 sha256=36f364b293a6cd2bc15cb7eb83dbee7ffd3b02f85a4e00f208f4f9374f72e8d5
  Stored in directory: /root/.cache/pip/wheels/92/f1/2b/3b46d54b134259f58c8363568569053248040859b1a145b3ce
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169

## Loading the Dataset

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import kneighbors_graph

# Load the Adult Income dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
column_names = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain',
    'capital-loss', 'hours-per-week', 'native-country', 'income'
]
df = pd.read_csv(url, header=None, names=column_names, na_values=' ?', skipinitialspace=True)
df.dropna(inplace=True)

# Encoding categorical variables
label_encoders = {}
for column in df.select_dtypes(include=['object']).columns:
    if column != 'income':
        label_encoders[column] = LabelEncoder()
        df[column] = label_encoders[column].fit_transform(df[column])

# Encode the target variable
df['income'] = df['income'].apply(lambda x: 1 if x == '>50K' else 0)

# Sensitive attribute
df['gender'] = df['sex']

# Define features and labels
features = df.drop(columns=['income'])
labels = df['income']

# Standardize the features
scaler = StandardScaler()
features = scaler.fit_transform(features)

# Convert to torch tensors
features = torch.tensor(features, dtype=torch.float)
labels = torch.tensor(labels.values, dtype=torch.long)

# Create k-NN graph
k = 5
knn_graph = kneighbors_graph(features, k, mode='connectivity', include_self=True)
edge_index = torch.tensor(knn_graph.nonzero(), dtype=torch.long)

# Prepare data object for PyTorch Geometric
data = Data(x=features, edge_index=edge_index, y=labels)

# Split data into training, validation, and test sets
indices = np.arange(data.num_nodes)
train_mask, temp_mask = train_test_split(indices, test_size=0.4, random_state=42)
val_mask, test_mask = train_test_split(temp_mask, test_size=0.5, random_state=42)

train_mask = torch.tensor(train_mask, dtype=torch.long)
val_mask = torch.tensor(val_mask, dtype=torch.long)
test_mask = torch.tensor(test_mask, dtype=torch.long)

# Display the sizes of the splits
print(f"Training set size: {len(train_mask)}")
print(f"Validation set size: {len(val_mask)}")
print(f"Test set size: {len(test_mask)}")

# Define the GCN model
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Initialize the model
input_dim = data.num_node_features
hidden_dim = 16
output_dim = len(labels.unique())
model = GCN(input_dim, hidden_dim, output_dim)

# Define optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.NLLLoss()

# Training function
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data)[train_mask]
    loss = criterion(out, data.y[train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

# Evaluation function
def evaluate(mask):
    model.eval()
    out = model(data)
    logits = out[mask]
    preds = logits.max(1)[1]
    acc = preds.eq(data.y[mask]).sum().item() / mask.size(0)
    return acc, criterion(logits, data.y[mask]).item(), preds

# Train the model with detailed logs
epochs = 200
for epoch in range(epochs):
    train_loss = train()
    train_acc, train_eval_loss, _ = evaluate(train_mask)
    val_acc, val_loss, _ = evaluate(val_mask)

    if epoch % 10 == 0:
        print(f"Epoch {epoch:03d}, Train Loss: {train_loss:.4f}, Train Eval Loss: {train_eval_loss:.4f}, Train Accuracy: {train_acc:.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")

# Final test accuracy and loss
test_acc, test_loss, test_preds = evaluate(test_mask)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")

<ipython-input-3-5ee4278b0208>:49: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  edge_index = torch.tensor(knn_graph.nonzero(), dtype=torch.long)


Training set size: 19536
Validation set size: 6512
Test set size: 6513
Epoch 000, Train Loss: 1.1283, Train Eval Loss: 0.9411, Train Accuracy: 0.4566, Validation Loss: 0.9412, Validation Accuracy: 0.4555
Epoch 010, Train Loss: 0.5070, Train Eval Loss: 0.4608, Train Accuracy: 0.7921, Validation Loss: 0.4678, Validation Accuracy: 0.7908
Epoch 020, Train Loss: 0.4428, Train Eval Loss: 0.4116, Train Accuracy: 0.8185, Validation Loss: 0.4121, Validation Accuracy: 0.8159
Epoch 030, Train Loss: 0.4140, Train Eval Loss: 0.3869, Train Accuracy: 0.8252, Validation Loss: 0.3868, Validation Accuracy: 0.8223
Epoch 040, Train Loss: 0.3909, Train Eval Loss: 0.3718, Train Accuracy: 0.8283, Validation Loss: 0.3737, Validation Accuracy: 0.8275
Epoch 050, Train Loss: 0.3827, Train Eval Loss: 0.3633, Train Accuracy: 0.8333, Validation Loss: 0.3655, Validation Accuracy: 0.8263
Epoch 060, Train Loss: 0.3766, Train Eval Loss: 0.3556, Train Accuracy: 0.8365, Validation Loss: 0.3578, Validation Accuracy: 0.830

## Fairness Matrix

In [ ]:
class Fairness:
    def __init__(self, df_profile, test_nodes_idx, targets, predictions, sens_attr):
        self.sens_attr = sens_attr
        self.df_profile = df_profile
        self.test_nodes_idx = np.array(test_nodes_idx)
        self.true_y = targets
        self.pred_y = predictions
        self.sens_attr_array = self.df_profile[self.sens_attr].values
        self.sens_attr_values = self.sens_attr_array[self.test_nodes_idx]
        self.s0 = self.sens_attr_values == 0
        self.s1 = self.sens_attr_values == 1
        self.y1_s0 = np.bitwise_and(self.true_y==1, self.s0)
        self.y1_s1 = np.bitwise_and(self.true_y==1, self.s1)
        self.y0_s0 = np.bitwise_and(self.true_y==0, self.s0)
        self.y0_s1 = np.bitwise_and(self.true_y==0, self.s1)

    def statistical_parity(self):
        if sum(self.s0) == 0 or sum(self.s1) == 0:
            print("Statistical Parity Difference (SPD): Undefined (division by zero)")
            return
        stat_parity = abs(sum(self.pred_y[self.s0]) / sum(self.s0) - sum(self.pred_y[self.s1]) / sum(self.s1))
        print("Statistical Parity Difference (SPD): {:.4f}".format(stat_parity))

    def equal_opportunity(self):
        if sum(self.y1_s0) == 0 or sum(self.y1_s1) == 0:
            print("Equal Opportunity Difference (EOD): Undefined (division by zero)")
            return
        equal_opp = abs(sum(self.pred_y[self.y1_s0]) / sum(self.y1_s0) - sum(self.pred_y[self.y1_s1]) / sum(self.y1_s1))
        print("Equal Opportunity Difference (EOD): {:.4f}".format(equal_opp))

    def overall_accuracy_equality(self):
        if sum(self.y0_s0) == 0 or sum(self.y1_s0) == 0 or sum(self.y0_s1) == 0 or sum(self.y1_s1) == 0:
            print("Overall Accuracy Equality Difference (OAED): Undefined (division by zero)")
            return
        oae_s0 = np.count_nonzero(self.pred_y[self.y0_s0]==0) / sum(self.y0_s0) + sum(self.pred_y[self.y1_s0]) / sum(self.y1_s0)
        oae_s1 = np.count_nonzero(self.pred_y[self.y0_s1]==0) / sum(self.y0_s1) + sum(self.pred_y[self.y1_s1]) / sum(self.y1_s1)
        oae_diff = abs(oae_s0 - oae_s1)
        print("Overall Accuracy Equality Difference (OAED): {:.4f}".format(oae_diff))

    def treatment_equality(self):
        if sum(self.y0_s0) == 0 or sum(self.y1_s0) == 0 or sum(self.y0_s1) == 0 or sum(self.y1_s1) == 0:
            print("Treatment Equality Difference (TED): Undefined (division by zero)")
            return
        te_s0 = (sum(self.pred_y[self.y0_s0]) / sum(self.y0_s0)) / (np.count_nonzero(self.pred_y[self.y1_s0]==0) / sum(self.y1_s0))
        te_s1 = (sum(self.pred_y[self.y0_s1]) / sum(self.y0_s1)) / (np.count_nonzero(self.pred_y[self.y1_s1]==0) / sum(self.y1_s1))
        te_diff_1 = abs(te_s0 - te_s1)
        te_s0 = (np.count_nonzero(self.pred_y[self.y1_s0]==0) / sum(self.y1_s0)) / (sum(self.pred_y[self.y0_s0]) / sum(self.y0_s0))
        te_s1 = (np.count_nonzero(self.pred_y[self.y1_s1]==0) / sum(self.y1_s1)) / (sum(self.pred_y[self.y0_s1]) / sum(self.y0_s1))
        te_diff_0 = abs(te_s0 - te_s1)
        te_diff = min(te_diff_1, te_diff_0)
        print("Treatment Equality Difference (TED): {:.4f}".format(te_diff))



In [ ]:
# 'gender' attribute is in the feature set
df_profile = pd.DataFrame(data.x.numpy(), columns=[f'feat_{i}' for i in range(data.num_node_features)])
df_profile['gender'] = np.random.randint(0, 2, size=data.num_nodes)
df_profile['Target'] = data.y.numpy()

# Evaluate fairness
test_labels = data.y[test_mask].numpy()
test_predictions = test_preds.numpy()
fairness = Fairness(df_profile, test_mask.numpy(), test_labels, test_predictions, 'gender')

# Compute and print fairness metrics
fairness.statistical_parity()
fairness.equal_opportunity()
fairness.overall_accuracy_equality()
fairness.treatment_equality()

Statistical Parity Difference (SPD): 0.0017
Equal Opportunity Difference (EOD): 0.0108
Overall Accuracy Equality Difference (OAED): 0.0070
Treatment Equality Difference (TED): 0.0112


##Reweighting (GCN)

In [ ]:
import numpy as np
import torch
from sklearn.utils import compute_class_weight

# --- Reweighting ---

# Calculate weights for reweighting
sample_weights = compute_class_weight('balanced', classes=np.unique(data.y[train_mask].cpu().numpy()), y=data.y[train_mask].cpu().numpy())
sample_weights = torch.tensor(sample_weights, dtype=torch.float32)

# Reweighted loss function
criterion_reweighted = torch.nn.NLLLoss(weight=sample_weights)

# Initialize a new GCN model for reweighting
model_reweighted = GCN(input_dim, hidden_dim, output_dim)

# Define optimizer
optimizer_reweighted = torch.optim.Adam(model_reweighted.parameters(), lr=0.01)

# Training function with reweighting
def train_reweighted():
    model_reweighted.train()
    optimizer_reweighted.zero_grad()
    out = model_reweighted(data)[train_mask]
    loss = criterion_reweighted(out, data.y[train_mask])
    loss.backward()
    optimizer_reweighted.step()
    return loss.item()

# Train the reweighted model
epochs = 200
for epoch in range(epochs):
    train_loss = train_reweighted()
    if epoch % 10 == 0:
        print(f"Epoch {epoch:03d}, Train Loss (Reweighted): {train_loss:.4f}")

# Evaluate the reweighted model
model_reweighted.eval()
out = model_reweighted(data)
logits = out[test_mask]
preds_reweighted = logits.max(1)[1]
test_acc_reweighted = preds_reweighted.eq(data.y[test_mask]).sum().item() / test_mask.size(0)
print(f"Test Accuracy (Reweighted): {test_acc_reweighted:.4f}")

# Create the df_profile DataFrame from data
df_profile = pd.DataFrame(data.x.cpu().numpy())
df_profile['Target'] = data.y.cpu().numpy()

df_profile['gender'] = (df_profile.index % 2).astype(int)  # sensitive attribute

# Re-evaluate fairness with the reweighted model
fairness_reweighted = Fairness(df_profile, test_mask.numpy(), data.y[test_mask].cpu().numpy(), preds_reweighted.cpu().numpy(), 'gender')
fairness_reweighted.statistical_parity()
fairness_reweighted.equal_opportunity()
fairness_reweighted.overall_accuracy_equality()
fairness_reweighted.treatment_equality()


Epoch 000, Train Loss (Reweighted): 0.9691
Epoch 010, Train Loss (Reweighted): 0.5905
Epoch 020, Train Loss (Reweighted): 0.5163
Epoch 030, Train Loss (Reweighted): 0.4932
Epoch 040, Train Loss (Reweighted): 0.4731
Epoch 050, Train Loss (Reweighted): 0.4607
Epoch 060, Train Loss (Reweighted): 0.4488
Epoch 070, Train Loss (Reweighted): 0.4417
Epoch 080, Train Loss (Reweighted): 0.4337
Epoch 090, Train Loss (Reweighted): 0.4317
Epoch 100, Train Loss (Reweighted): 0.4314
Epoch 110, Train Loss (Reweighted): 0.4269
Epoch 120, Train Loss (Reweighted): 0.4264
Epoch 130, Train Loss (Reweighted): 0.4256
Epoch 140, Train Loss (Reweighted): 0.4265
Epoch 150, Train Loss (Reweighted): 0.4261
Epoch 160, Train Loss (Reweighted): 0.4230
Epoch 170, Train Loss (Reweighted): 0.4238
Epoch 180, Train Loss (Reweighted): 0.4228
Epoch 190, Train Loss (Reweighted): 0.4249
Test Accuracy (Reweighted): 0.7835
Statistical Parity Difference (SPD): 0.0115
Equal Opportunity Difference (EOD): 0.0265
Overall Accuracy E

## Reject Option classifier (GCN)

In [ ]:
## Reject Option Classifier ##

threshold = 0.6  # Example threshold, you might need to tune this

# Initialize a new GCN model for reject option classification
model_reject = GCN(input_dim, hidden_dim, output_dim)

# Define optimizer
optimizer_reject = torch.optim.Adam(model_reject.parameters(), lr=0.01)

# Training function (same as the original training function)
def train_reject():
    model_reject.train()
    optimizer_reject.zero_grad()
    out = model_reject(data)[train_mask]
    loss = criterion(out, data.y[train_mask])
    loss.backward()
    optimizer_reject.step()
    return loss.item()

# Train the model for reject option classification
epochs = 200
for epoch in range(epochs):
    train_loss = train_reject()
    if epoch % 10 == 0:
        print(f"Epoch {epoch:03d}, Train Loss (Reject Option): {train_loss:.4f}")

# Evaluation function with reject option
def evaluate_reject(mask):
    model_reject.eval()
    out = model_reject(data)
    logits = out[mask]
    probs = F.softmax(logits, dim=1)

    # Get predicted classes
    preds = logits.max(1)[1]

    # Identify instances to reject (where the maximum probability is below the threshold)
    reject_mask = probs.max(1)[0] < threshold

    # Calculate accuracy on non-rejected instances
    non_rejected_mask = ~reject_mask
    num_non_rejected = non_rejected_mask.sum().item()
    if num_non_rejected > 0:
        acc = preds[non_rejected_mask].eq(data.y[mask][non_rejected_mask]).sum().item() / num_non_rejected
    else:
        acc = 0  # Handle the case where all instances are rejected

    return acc, preds, reject_mask

# Evaluate the model with reject option
test_acc_reject, test_preds_reject, test_reject_mask = evaluate_reject(test_mask)
print(f"Test Accuracy (Reject Option): {test_acc_reject:.4f}")
print(f"Rejection Rate: {test_reject_mask.sum().item() / test_mask.size(0):.4f}")

# (Only evaluating the fairness on the non-rejected instances)
fairness_reject = Fairness(df_profile, test_mask.numpy()[~test_reject_mask.cpu().numpy()],
                            data.y[test_mask][~test_reject_mask].cpu().numpy(),
                            test_preds_reject[~test_reject_mask].cpu().numpy(), 'gender')
fairness_reject.statistical_parity()
fairness_reject.equal_opportunity()
fairness_reject.overall_accuracy_equality()
fairness_reject.treatment_equality()

# Print the test accuracies of the original and reject option models
print(f"Test Accuracy (Original): {test_acc:.4f}")
print(f"Test Accuracy (Reject Option): {test_acc_reject:.4f}")



Epoch 000, Train Loss (Reject Option): 0.7560
Epoch 010, Train Loss (Reject Option): 0.4707
Epoch 020, Train Loss (Reject Option): 0.4258
Epoch 030, Train Loss (Reject Option): 0.4021
Epoch 040, Train Loss (Reject Option): 0.3905
Epoch 050, Train Loss (Reject Option): 0.3788
Epoch 060, Train Loss (Reject Option): 0.3703
Epoch 070, Train Loss (Reject Option): 0.3660
Epoch 080, Train Loss (Reject Option): 0.3606
Epoch 090, Train Loss (Reject Option): 0.3598
Epoch 100, Train Loss (Reject Option): 0.3548
Epoch 110, Train Loss (Reject Option): 0.3546
Epoch 120, Train Loss (Reject Option): 0.3529
Epoch 130, Train Loss (Reject Option): 0.3515
Epoch 140, Train Loss (Reject Option): 0.3499
Epoch 150, Train Loss (Reject Option): 0.3512
Epoch 160, Train Loss (Reject Option): 0.3515
Epoch 170, Train Loss (Reject Option): 0.3502
Epoch 180, Train Loss (Reject Option): 0.3489
Epoch 190, Train Loss (Reject Option): 0.3476
Test Accuracy (Reject Option): 0.8844
Rejection Rate: 0.1317
Statistical Parity 

## Prejudice Remover Regularizer (GCN)


In [ ]:
# --- Prejudice Remover Regularizer (PRR) ---

def calculate_prr_term(model, data, mask, sensitive_attr):
    out = model(data)[mask]
    probs = F.softmax(out, dim=1)

    # Calculate the mean probabilities for each sensitive attribute value
    mean_probs_0 = probs[torch.tensor(df_profile[sensitive_attr].values[mask.numpy()] == 0, dtype=torch.bool)].mean(dim=0)
    mean_probs_1 = probs[torch.tensor(df_profile[sensitive_attr].values[mask.numpy()] == 1, dtype=torch.bool)].mean(dim=0)

    # Calculate the PRR term (squared Euclidean distance between mean probabilities)
    prr_term = torch.sum((mean_probs_0 - mean_probs_1) ** 2)
    return prr_term

# Training function with PRR
def train_prr(eta=0.1):  # eta controls the strength of the PRR term
    model.train()
    optimizer.zero_grad()

    loss_main = criterion(model(data)[train_mask], data.y[train_mask])

    loss_prr = 0
    for attr in ['gender']: # Assuming 'gender' is the sensitive attribute
        loss_prr += calculate_prr_term(model, data, train_mask, attr)

    total_loss = loss_main + eta * loss_prr
    total_loss.backward()
    optimizer.step()
    return total_loss.item(), loss_main.item(), loss_prr.item()

# Train the model with PRR
epochs = 200
for epoch in range(epochs):
    loss, loss_main, loss_prr = train_prr()
    if epoch % 10 == 0:
        print(f"Epoch {epoch:03d}, Total Loss: {loss:.4f}, Main Task Loss: {loss_main:.4f}, PRR Loss: {loss_prr:.4f}")

# Evaluate the model after PRR mitigation
model.eval()
out = model(data)
logits = out[test_mask]
preds_prr = logits.max(1)[1]
test_acc_prr = preds_prr.eq(data.y[test_mask]).sum().item() / test_mask.size(0)
print(f"Test Accuracy (After PRR): {test_acc_prr:.4f}")

# Re-evaluate fairness after PRR mitigation
fairness_prr = Fairness(df_profile, test_mask.numpy(), data.y[test_mask].cpu().numpy(), preds_prr.cpu().numpy(), 'gender')
fairness_prr.statistical_parity()
fairness_prr.equal_opportunity()
fairness_prr.overall_accuracy_equality()
fairness_prr.treatment_equality()

# --- Comparison ---
print("\n--- Comparison ---")
print("Original Model Test Accuracy: {:.4f}".format(test_acc))
print("Reweighted Model Test Accuracy: {:.4f}".format(test_acc_reweighted))
print("PRR Model Test Accuracy: {:.4f}".format(test_acc_prr))
print("Reject Option Model Test Accuracy: {:.4f}".format(test_acc_reject))
print("PRR Model Rejection Rate: {:.4f}".format(test_reject_mask.sum().item() / test_mask.size(0)))  # Print the rejection rate

Epoch 000, Total Loss: 0.3500, Main Task Loss: 0.3500, PRR Loss: 0.0000
Epoch 010, Total Loss: 0.3507, Main Task Loss: 0.3507, PRR Loss: 0.0000
Epoch 020, Total Loss: 0.3499, Main Task Loss: 0.3499, PRR Loss: 0.0000
Epoch 030, Total Loss: 0.3501, Main Task Loss: 0.3501, PRR Loss: 0.0000
Epoch 040, Total Loss: 0.3479, Main Task Loss: 0.3479, PRR Loss: 0.0000
Epoch 050, Total Loss: 0.3500, Main Task Loss: 0.3500, PRR Loss: 0.0000
Epoch 060, Total Loss: 0.3476, Main Task Loss: 0.3476, PRR Loss: 0.0000
Epoch 070, Total Loss: 0.3475, Main Task Loss: 0.3475, PRR Loss: 0.0000
Epoch 080, Total Loss: 0.3503, Main Task Loss: 0.3503, PRR Loss: 0.0000
Epoch 090, Total Loss: 0.3502, Main Task Loss: 0.3502, PRR Loss: 0.0000
Epoch 100, Total Loss: 0.3487, Main Task Loss: 0.3487, PRR Loss: 0.0000
Epoch 110, Total Loss: 0.3472, Main Task Loss: 0.3472, PRR Loss: 0.0000
Epoch 120, Total Loss: 0.3483, Main Task Loss: 0.3483, PRR Loss: 0.0000
Epoch 130, Total Loss: 0.3483, Main Task Loss: 0.3483, PRR Loss:

## Rich Subgroup Fairness (GCN)

In [ ]:
# --- Rich Subgroup Fairness (RSF) ---

sensitive_attr = 'gender'

# Calculate the RSF term
def calculate_rsf_term(model, data, mask, sensitive_attr, gamma=0.1):
    out = model(data)[mask]
    probs = F.softmax(out, dim=1)

    # Calculate the empirical risk for each subgroup
    risk_0 = F.nll_loss(probs[torch.tensor(df_profile[sensitive_attr].values[mask.numpy()] == 0, dtype=torch.bool)],
                       data.y[mask][torch.tensor(df_profile[sensitive_attr].values[mask.numpy()] == 0, dtype=torch.bool)])
    risk_1 = F.nll_loss(probs[torch.tensor(df_profile[sensitive_attr].values[mask.numpy()] == 1, dtype=torch.bool)],
                       data.y[mask][torch.tensor(df_profile[sensitive_attr].values[mask.numpy()] == 1, dtype=torch.bool)])

    # Calculate the RSF term
    rsf_term = torch.abs(risk_0 - risk_1) - gamma * (risk_0 + risk_1)
    return rsf_term

# Initialize a new GCN model for RSF
model_rsf = GCN(input_dim, hidden_dim, output_dim)

# Define optimizer
optimizer_rsf = torch.optim.Adam(model_rsf.parameters(), lr=0.01)

# Training function with RSF
def train_rsf(lambda_=0.1):  # lambda_ controls the strength of the RSF term
    model_rsf.train()
    optimizer_rsf.zero_grad()

    loss_main = criterion(model_rsf(data)[train_mask], data.y[train_mask])

    loss_rsf = calculate_rsf_term(model_rsf, data, train_mask, sensitive_attr)

    total_loss = loss_main + lambda_ * loss_rsf
    total_loss.backward()
    optimizer_rsf.step()
    return total_loss.item(), loss_main.item(), loss_rsf.item()

# Train the model with RSF
epochs = 200
for epoch in range(epochs):
    loss, loss_main, loss_rsf = train_rsf()
    if epoch % 10 == 0:
        print(f"Epoch {epoch:03d}, Total Loss: {loss:.4f}, Main Task Loss: {loss_main:.4f}, RSF Loss: {loss_rsf:.4f}")

# Evaluate the model after RSF mitigation
model_rsf.eval()
out = model_rsf(data)
logits = out[test_mask]
preds_rsf = logits.max(1)[1]
test_acc_rsf = preds_rsf.eq(data.y[test_mask]).sum().item() / test_mask.size(0)
print(f"Test Accuracy (After RSF): {test_acc_rsf:.4f}")

# Re-evaluate fairness after RSF mitigation
fairness_rsf = Fairness(df_profile, test_mask.numpy(), data.y[test_mask].cpu().numpy(), preds_rsf.cpu().numpy(), 'gender')
fairness_rsf.statistical_parity()
fairness_rsf.equal_opportunity()
fairness_rsf.overall_accuracy_equality()
fairness_rsf.treatment_equality()

# --- Comparison ---
print("\n--- Comparison ---")
print("Original Model Test Accuracy: {:.4f}".format(test_acc))
print("Reweighted Model Test Accuracy: {:.4f}".format(test_acc_reweighted))
print("RSF Model Test Accuracy: {:.4f}".format(test_acc_rsf))
print("PRR Model Test Accuracy: {:.4f}".format(test_acc_prr))
print("Reject Option Model Test Accuracy: {:.4f}".format(test_acc_reject))
print("PRR Model Rejection Rate: {:.4f}".format(test_reject_mask.sum().item() / test_mask.size(0)))  # Print the rejection rate

Epoch 000, Total Loss: 0.7325, Main Task Loss: 0.7207, RSF Loss: 0.1183
Epoch 010, Total Loss: 0.4602, Main Task Loss: 0.4454, RSF Loss: 0.1485
Epoch 020, Total Loss: 0.4193, Main Task Loss: 0.4040, RSF Loss: 0.1530
Epoch 030, Total Loss: 0.4023, Main Task Loss: 0.3872, RSF Loss: 0.1507
Epoch 040, Total Loss: 0.3902, Main Task Loss: 0.3751, RSF Loss: 0.1518
Epoch 050, Total Loss: 0.3816, Main Task Loss: 0.3663, RSF Loss: 0.1524
Epoch 060, Total Loss: 0.3731, Main Task Loss: 0.3577, RSF Loss: 0.1536
Epoch 070, Total Loss: 0.3727, Main Task Loss: 0.3573, RSF Loss: 0.1541
Epoch 080, Total Loss: 0.3702, Main Task Loss: 0.3548, RSF Loss: 0.1540
Epoch 090, Total Loss: 0.3700, Main Task Loss: 0.3545, RSF Loss: 0.1556
Epoch 100, Total Loss: 0.3683, Main Task Loss: 0.3529, RSF Loss: 0.1543
Epoch 110, Total Loss: 0.3686, Main Task Loss: 0.3532, RSF Loss: 0.1547
Epoch 120, Total Loss: 0.3689, Main Task Loss: 0.3535, RSF Loss: 0.1547
Epoch 130, Total Loss: 0.3689, Main Task Loss: 0.3533, RSF Loss:

##Comparing Results (GCN)

In [ ]:
import pandas as pd
# Store results in a dictionary
results = {
    'Model': ['Original', 'Reweighted', 'PRR', 'Reject Option', 'RSF'],
    'Test Accuracy': [test_acc, test_acc_reweighted, test_acc_prr, test_acc_reject, test_acc_rsf],
    'Rejection Rate': [0, 0, 0, test_reject_mask.sum().item() / test_mask.size(0), 0],  # Add rejection rate for Reject Option
    'Statistical Parity': [fairness.statistical_parity(), fairness_reweighted.statistical_parity(),
                           fairness_prr.statistical_parity(), fairness_reject.statistical_parity(),
                           fairness_rsf.statistical_parity()],
    'Equal Opportunity': [fairness.equal_opportunity(), fairness_reweighted.equal_opportunity(),
                          fairness_prr.equal_opportunity(), fairness_reject.equal_opportunity(),
                          fairness_rsf.equal_opportunity()],
    'Overall Accuracy Equality': [fairness.overall_accuracy_equality(), fairness_reweighted.overall_accuracy_equality(),
                                 fairness_prr.overall_accuracy_equality(), fairness_reject.overall_accuracy_equality(),
                                 fairness_rsf.overall_accuracy_equality()],
    'Treatment Equality': [fairness.treatment_equality(), fairness_reweighted.treatment_equality(),
                           fairness_prr.treatment_equality(), fairness_reject.treatment_equality(),
                           fairness_rsf.treatment_equality()]
}

# Create a Pandas DataFrame from the results
df_results = pd.DataFrame(results)

# Display the DataFrame as a table
display(df_results)


Statistical Parity Difference (SPD): 0.0017
Statistical Parity Difference (SPD): 0.0115
Statistical Parity Difference (SPD): 0.0054
Statistical Parity Difference (SPD): 0.0003
Statistical Parity Difference (SPD): 0.0060
Equal Opportunity Difference (EOD): 0.0108
Equal Opportunity Difference (EOD): 0.0265
Equal Opportunity Difference (EOD): 0.0328
Equal Opportunity Difference (EOD): 0.0370
Equal Opportunity Difference (EOD): 0.0353
Overall Accuracy Equality Difference (OAED): 0.0070
Overall Accuracy Equality Difference (OAED): 0.0204
Overall Accuracy Equality Difference (OAED): 0.0364
Overall Accuracy Equality Difference (OAED): 0.0369
Overall Accuracy Equality Difference (OAED): 0.0389
Treatment Equality Difference (TED): 0.0112
Treatment Equality Difference (TED): 0.1268
Treatment Equality Difference (TED): 0.0014
Treatment Equality Difference (TED): 0.0071
Treatment Equality Difference (TED): 0.0030


,Model,Test Accuracy,Rejection Rate,Statistical Parity,Equal Opportunity,Overall Accuracy Equality,Treatment Equality
0,Original,0.844311,0.000000,None,None,None,None
1,Reweighted,0.783510,0.000000,None,None,None,None
2,PRR,0.845693,0.000000,None,None,None,None
3,Reject Option,0.884350,0.131737,None,None,None,None
4,RSF,0.846000,0.000000,None,None,None,None


# GAT Model

In [ ]:
from torch_geometric.nn import GATConv

# Define the GAT model
class GAT(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GAT, self).__init__()
        self.conv1 = GATConv(input_dim, hidden_dim, heads=8, dropout=0.6)
        self.conv2 = GATConv(hidden_dim * 8, output_dim, heads=1, concat=False, dropout=0.6)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Initialize the GAT model
model_gat = GAT(input_dim, hidden_dim, output_dim)

# Define optimizer
optimizer_gat = torch.optim.Adam(model_gat.parameters(), lr=0.01)

# Training function
def train_gat():
    model_gat.train()
    optimizer_gat.zero_grad()
    out = model_gat(data)[train_mask]
    loss = criterion(out, data.y[train_mask])
    loss.backward()
    optimizer_gat.step()
    return loss.item()

# Train the GAT model
epochs = 200
for epoch in range(epochs):
    train_loss = train_gat()
    if epoch % 10 == 0:
        print(f"Epoch {epoch:03d}, Train Loss (GAT): {train_loss:.4f}")

# Evaluate the GAT model
model_gat.eval()
out = model_gat(data)
logits = out[test_mask]
preds_gat = logits.max(1)[1]
test_acc_gat = preds_gat.eq(data.y[test_mask]).sum().item() / test_mask.size(0)
print(f"Test Accuracy (GAT): {test_acc_gat:.4f}")

# Evaluate fairness for the GAT model
fairness_gat = Fairness(df_profile, test_mask.numpy(), data.y[test_mask].cpu().numpy(), preds_gat.cpu().numpy(), 'gender')
fairness_gat.statistical_parity()
fairness_gat.equal_opportunity()
fairness_gat.overall_accuracy_equality()
fairness_gat.treatment_equality()


Epoch 000, Train Loss (GAT): 0.8740
Epoch 010, Train Loss (GAT): 0.5112
Epoch 020, Train Loss (GAT): 0.4752
Epoch 030, Train Loss (GAT): 0.4611
Epoch 040, Train Loss (GAT): 0.4502
Epoch 050, Train Loss (GAT): 0.4507
Epoch 060, Train Loss (GAT): 0.4434
Epoch 070, Train Loss (GAT): 0.4423
Epoch 080, Train Loss (GAT): 0.4371
Epoch 090, Train Loss (GAT): 0.4322
Epoch 100, Train Loss (GAT): 0.4349
Epoch 110, Train Loss (GAT): 0.4370
Epoch 120, Train Loss (GAT): 0.4386
Epoch 130, Train Loss (GAT): 0.4286
Epoch 140, Train Loss (GAT): 0.4340
Epoch 150, Train Loss (GAT): 0.4319
Epoch 160, Train Loss (GAT): 0.4339
Epoch 170, Train Loss (GAT): 0.4310
Epoch 180, Train Loss (GAT): 0.4291
Epoch 190, Train Loss (GAT): 0.4308
Test Accuracy (GAT): 0.8283
Statistical Parity Difference (SPD): 0.0044
Equal Opportunity Difference (EOD): 0.0075
Overall Accuracy Equality Difference (OAED): 0.0036
Treatment Equality Difference (TED): 0.0077


## Reweighting (GAT)

In [ ]:
import numpy as np
# --- Reweighting on GAT ---

# Calculate weights
sample_weights = compute_class_weight('balanced', classes=np.unique(data.y[train_mask].cpu().numpy()), y=data.y[train_mask].cpu().numpy())
sample_weights = torch.tensor(sample_weights, dtype=torch.float32)

# Reweighted loss function
criterion_reweighted = torch.nn.NLLLoss(weight=sample_weights)

# Initialize a new GAT model for reweighting
model_gat_reweighted = GAT(input_dim, hidden_dim, output_dim)

# Define optimizer
optimizer_gat_reweighted = torch.optim.Adam(model_gat_reweighted.parameters(), lr=0.01)

# Training function with reweighting for GAT
def train_gat_reweighted():
    model_gat_reweighted.train()
    optimizer_gat_reweighted.zero_grad()
    out = model_gat_reweighted(data)[train_mask]
    loss = criterion_reweighted(out, data.y[train_mask])
    loss.backward()
    optimizer_gat_reweighted.step()
    return loss.item()

# Train the reweighted GAT model
epochs = 200
for epoch in range(epochs):
    train_loss = train_gat_reweighted()
    if epoch % 10 == 0:
        print(f"Epoch {epoch:03d}, Train Loss (GAT Reweighted): {train_loss:.4f}")

# Evaluate the reweighted GAT model
model_gat_reweighted.eval()
out = model_gat_reweighted(data)
logits = out[test_mask]
preds_gat_reweighted = logits.max(1)[1]
test_acc_gat_reweighted = preds_gat_reweighted.eq(data.y[test_mask]).sum().item() / test_mask.size(0)
print(f"Test Accuracy (GAT Reweighted): {test_acc_gat_reweighted:.4f}")

# Evaluate fairness for the reweighted GAT model
fairness_gat_reweighted = Fairness(df_profile, test_mask.numpy(), data.y[test_mask].cpu().numpy(), preds_gat_reweighted.cpu().numpy(), 'gender')
fairness_gat_reweighted.statistical_parity()
fairness_gat_reweighted.equal_opportunity()
fairness_gat_reweighted.overall_accuracy_equality()
fairness_gat_reweighted.treatment_equality()


Epoch 000, Train Loss (GAT Reweighted): 1.0302
Epoch 010, Train Loss (GAT Reweighted): 0.6207
Epoch 020, Train Loss (GAT Reweighted): 0.5757
Epoch 030, Train Loss (GAT Reweighted): 0.5622
Epoch 040, Train Loss (GAT Reweighted): 0.5536
Epoch 050, Train Loss (GAT Reweighted): 0.5515
Epoch 060, Train Loss (GAT Reweighted): 0.5407
Epoch 070, Train Loss (GAT Reweighted): 0.5378
Epoch 080, Train Loss (GAT Reweighted): 0.5411
Epoch 090, Train Loss (GAT Reweighted): 0.5381
Epoch 100, Train Loss (GAT Reweighted): 0.5358
Epoch 110, Train Loss (GAT Reweighted): 0.5366
Epoch 120, Train Loss (GAT Reweighted): 0.5301
Epoch 130, Train Loss (GAT Reweighted): 0.5375
Epoch 140, Train Loss (GAT Reweighted): 0.5317
Epoch 150, Train Loss (GAT Reweighted): 0.5306
Epoch 160, Train Loss (GAT Reweighted): 0.5263
Epoch 170, Train Loss (GAT Reweighted): 0.5328
Epoch 180, Train Loss (GAT Reweighted): 0.5354
Epoch 190, Train Loss (GAT Reweighted): 0.5259
Test Accuracy (GAT Reweighted): 0.7095
Statistical Parity Di

## Reject Option Classifier (GAT)

In [ ]:
# --- Reject Option Classifier on GAT ---

threshold = 0.6

# Initialize a new GAT model for reject option classification
model_gat_reject = GAT(input_dim, hidden_dim, output_dim)

# Define optimizer
optimizer_gat_reject = torch.optim.Adam(model_gat_reject.parameters(), lr=0.01)

# Training function (same as the original training function for GAT)
def train_gat_reject():
    model_gat_reject.train()
    optimizer_gat_reject.zero_grad()
    out = model_gat_reject(data)[train_mask]
    loss = criterion(out, data.y[train_mask])
    loss.backward()
    optimizer_gat_reject.step()
    return loss.item()

# Train the GAT model for reject option classification
epochs = 200
for epoch in range(epochs):
    train_loss = train_gat_reject()
    if epoch % 10 == 0:
        print(f"Epoch {epoch:03d}, Train Loss (GAT Reject Option): {train_loss:.4f}")

# Evaluation function with reject option (same as before)
def evaluate_reject(mask):
    model_gat_reject.eval()
    out = model_gat_reject(data)
    logits = out[mask]
    probs = F.softmax(logits, dim=1)

    # Get predicted classes
    preds = logits.max(1)[1]

    # Identify instances to reject (where the maximum probability is below the threshold)
    reject_mask = probs.max(1)[0] < threshold

    # Calculate accuracy on non-rejected instances
    non_rejected_mask = ~reject_mask
    num_non_rejected = non_rejected_mask.sum().item()
    if num_non_rejected > 0:
        acc = preds[non_rejected_mask].eq(data.y[mask][non_rejected_mask]).sum().item() / num_non_rejected
    else:
        acc = 0  # Handle the case where all instances are rejected

    return acc, preds, reject_mask

# Evaluate the GAT model with reject option
test_acc_gat_reject, test_preds_gat_reject, test_reject_mask_gat = evaluate_reject(test_mask)
print(f"Test Accuracy (GAT Reject Option): {test_acc_gat_reject:.4f}")
print(f"Rejection Rate (GAT Reject Option): {test_reject_mask_gat.sum().item() / test_mask.size(0):.4f}")

# (Only evaluating the fairness on the non-rejected instances)
fairness_gat_reject = Fairness(df_profile, test_mask.numpy()[~test_reject_mask_gat.cpu().numpy()],
                            data.y[test_mask][~test_reject_mask_gat].cpu().numpy(),
                            test_preds_gat_reject[~test_reject_mask_gat].cpu().numpy(), 'gender')
fairness_gat_reject.statistical_parity()
fairness_gat_reject.equal_opportunity()
fairness_gat_reject.overall_accuracy_equality()
fairness_gat_reject.treatment_equality()


Epoch 000, Train Loss (GAT Reject Option): 1.2862
Epoch 010, Train Loss (GAT Reject Option): 0.5285
Epoch 020, Train Loss (GAT Reject Option): 0.4931
Epoch 030, Train Loss (GAT Reject Option): 0.4719
Epoch 040, Train Loss (GAT Reject Option): 0.4611
Epoch 050, Train Loss (GAT Reject Option): 0.4476
Epoch 060, Train Loss (GAT Reject Option): 0.4462
Epoch 070, Train Loss (GAT Reject Option): 0.4501
Epoch 080, Train Loss (GAT Reject Option): 0.4459
Epoch 090, Train Loss (GAT Reject Option): 0.4393
Epoch 100, Train Loss (GAT Reject Option): 0.4367
Epoch 110, Train Loss (GAT Reject Option): 0.4333
Epoch 120, Train Loss (GAT Reject Option): 0.4342
Epoch 130, Train Loss (GAT Reject Option): 0.4387
Epoch 140, Train Loss (GAT Reject Option): 0.4338
Epoch 150, Train Loss (GAT Reject Option): 0.4377
Epoch 160, Train Loss (GAT Reject Option): 0.4341
Epoch 170, Train Loss (GAT Reject Option): 0.4398
Epoch 180, Train Loss (GAT Reject Option): 0.4304
Epoch 190, Train Loss (GAT Reject Option): 0.4346


## Prejudice Remover Regularizer (GAT)

In [ ]:
# --- Prejudice Remover Regularizer on GAT ---

def calculate_prr_term_gat(model, data, mask, sensitive_attr):
    out = model(data)[mask]
    probs = F.softmax(out, dim=1)

    # Calculate the mean probabilities for each sensitive attribute value
    mean_probs_0 = probs[torch.tensor(df_profile[sensitive_attr].values[mask.numpy()] == 0, dtype=torch.bool)].mean(dim=0)
    mean_probs_1 = probs[torch.tensor(df_profile[sensitive_attr].values[mask.numpy()] == 1, dtype=torch.bool)].mean(dim=0)

    # Calculate the PRR term (squared Euclidean distance between mean probabilities)
    prr_term = torch.sum((mean_probs_0 - mean_probs_1) ** 2)
    return prr_term

# Training function with PRR for GAT
def train_gat_prr(eta=0.1):  # eta controls the strength of the PRR term
    model_gat_prr.train()
    optimizer_gat_prr.zero_grad()

    loss_main = criterion(model_gat_prr(data)[train_mask], data.y[train_mask])

    loss_prr = 0
    for attr in ['gender']:
        loss_prr += calculate_prr_term_gat(model_gat_prr, data, train_mask, attr)

    total_loss = loss_main + eta * loss_prr
    total_loss.backward()
    optimizer_gat_prr.step()
    return total_loss.item(), loss_main.item(), loss_prr.item()

# Initialize a new GAT model for PRR
model_gat_prr = GAT(input_dim, hidden_dim, output_dim)

# Define optimizer
optimizer_gat_prr = torch.optim.Adam(model_gat_prr.parameters(), lr=0.01)

# Train the GAT model with PRR
epochs = 200
for epoch in range(epochs):
    loss, loss_main, loss_prr = train_gat_prr()
    if epoch % 10 == 0:
        print(f"Epoch {epoch:03d}, Total Loss: {loss:.4f}, Main Task Loss: {loss_main:.4f}, PRR Loss: {loss_prr:.4f}")

# Evaluate the GAT model after PRR mitigation
model_gat_prr.eval()
out = model_gat_prr(data)
logits = out[test_mask]
preds_gat_prr = logits.max(1)[1]
test_acc_gat_prr = preds_gat_prr.eq(data.y[test_mask]).sum().item() / test_mask.size(0)
print(f"Test Accuracy (GAT After PRR): {test_acc_gat_prr:.4f}")

# Re-evaluate fairness after PRR mitigation for GAT
fairness_gat_prr = Fairness(df_profile, test_mask.numpy(), data.y[test_mask].cpu().numpy(), preds_gat_prr.cpu().numpy(), 'gender')
fairness_gat_prr.statistical_parity()
fairness_gat_prr.equal_opportunity()
fairness_gat_prr.overall_accuracy_equality()
fairness_gat_prr.treatment_equality()


Epoch 000, Total Loss: 1.0050, Main Task Loss: 1.0050, PRR Loss: 0.0000
Epoch 010, Total Loss: 0.5457, Main Task Loss: 0.5457, PRR Loss: 0.0000
Epoch 020, Total Loss: 0.4886, Main Task Loss: 0.4886, PRR Loss: 0.0000
Epoch 030, Total Loss: 0.4679, Main Task Loss: 0.4679, PRR Loss: 0.0000
Epoch 040, Total Loss: 0.4587, Main Task Loss: 0.4587, PRR Loss: 0.0000
Epoch 050, Total Loss: 0.4530, Main Task Loss: 0.4530, PRR Loss: 0.0000
Epoch 060, Total Loss: 0.4414, Main Task Loss: 0.4414, PRR Loss: 0.0000
Epoch 070, Total Loss: 0.4416, Main Task Loss: 0.4416, PRR Loss: 0.0000
Epoch 080, Total Loss: 0.4401, Main Task Loss: 0.4401, PRR Loss: 0.0001
Epoch 090, Total Loss: 0.4383, Main Task Loss: 0.4383, PRR Loss: 0.0000
Epoch 100, Total Loss: 0.4400, Main Task Loss: 0.4400, PRR Loss: 0.0000
Epoch 110, Total Loss: 0.4349, Main Task Loss: 0.4349, PRR Loss: 0.0000
Epoch 120, Total Loss: 0.4386, Main Task Loss: 0.4386, PRR Loss: 0.0000
Epoch 130, Total Loss: 0.4427, Main Task Loss: 0.4427, PRR Loss:

## Rich Subgroup Fairness (GAT)

In [ ]:
# --- Rich Subgroup Fairness (RSF) on GAT ---

# Calculate the RSF term for GAT
def calculate_rsf_term_gat(model, data, mask, sensitive_attr, gamma=0.1):
    out = model(data)[mask]
    probs = F.softmax(out, dim=1)

    # Calculate the empirical risk for each subgroup
    risk_0 = F.nll_loss(probs[torch.tensor(df_profile[sensitive_attr].values[mask.numpy()] == 0, dtype=torch.bool)],
                       data.y[mask][torch.tensor(df_profile[sensitive_attr].values[mask.numpy()] == 0, dtype=torch.bool)])
    risk_1 = F.nll_loss(probs[torch.tensor(df_profile[sensitive_attr].values[mask.numpy()] == 1, dtype=torch.bool)],
                       data.y[mask][torch.tensor(df_profile[sensitive_attr].values[mask.numpy()] == 1, dtype=torch.bool)])

    # Calculate the RSF term
    rsf_term = torch.abs(risk_0 - risk_1) - gamma * (risk_0 + risk_1)
    return rsf_term

# Initialize a new GAT model for RSF
model_gat_rsf = GAT(input_dim, hidden_dim, output_dim)

# Define optimizer
optimizer_gat_rsf = torch.optim.Adam(model_gat_rsf.parameters(), lr=0.01)

# Training function with RSF for GAT
def train_gat_rsf(lambda_=0.1):  # lambda_ controls the strength of the RSF term
    model_gat_rsf.train()
    optimizer_gat_rsf.zero_grad()

    loss_main = criterion(model_gat_rsf(data)[train_mask], data.y[train_mask])

    loss_rsf = calculate_rsf_term_gat(model_gat_rsf, data, train_mask, sensitive_attr)

    total_loss = loss_main + lambda_ * loss_rsf
    total_loss.backward()
    optimizer_gat_rsf.step()
    return total_loss.item(), loss_main.item(), loss_rsf.item()

# Train the GAT model with RSF
epochs = 200
for epoch in range(epochs):
    loss, loss_main, loss_rsf = train_gat_rsf()
    if epoch % 10 == 0:
        print(f"Epoch {epoch:03d}, Total Loss: {loss:.4f}, Main Task Loss: {loss_main:.4f}, RSF Loss: {loss_rsf:.4f}")

# Evaluate the GAT model after RSF mitigation
model_gat_rsf.eval()
out = model_gat_rsf(data)
logits = out[test_mask]
preds_gat_rsf = logits.max(1)[1]
test_acc_gat_rsf = preds_gat_rsf.eq(data.y[test_mask]).sum().item() / test_mask.size(0)
print(f"Test Accuracy (GAT After RSF): {test_acc_gat_rsf:.4f}")

# Re-evaluate fairness after RSF mitigation for GAT
fairness_gat_rsf = Fairness(df_profile, test_mask.numpy(), data.y[test_mask].cpu().numpy(), preds_gat_rsf.cpu().numpy(), 'gender')
fairness_gat_rsf.statistical_parity()
fairness_gat_rsf.equal_opportunity()
fairness_gat_rsf.overall_accuracy_equality()
fairness_gat_rsf.treatment_equality()


Epoch 000, Total Loss: 0.9970, Main Task Loss: 0.9862, RSF Loss: 0.1075
Epoch 010, Total Loss: 0.5255, Main Task Loss: 0.5118, RSF Loss: 0.1372
Epoch 020, Total Loss: 0.4923, Main Task Loss: 0.4789, RSF Loss: 0.1343
Epoch 030, Total Loss: 0.4798, Main Task Loss: 0.4657, RSF Loss: 0.1405
Epoch 040, Total Loss: 0.4733, Main Task Loss: 0.4592, RSF Loss: 0.1412
Epoch 050, Total Loss: 0.4647, Main Task Loss: 0.4503, RSF Loss: 0.1436
Epoch 060, Total Loss: 0.4613, Main Task Loss: 0.4472, RSF Loss: 0.1417
Epoch 070, Total Loss: 0.4552, Main Task Loss: 0.4410, RSF Loss: 0.1424
Epoch 080, Total Loss: 0.4568, Main Task Loss: 0.4424, RSF Loss: 0.1438
Epoch 090, Total Loss: 0.4489, Main Task Loss: 0.4344, RSF Loss: 0.1447
Epoch 100, Total Loss: 0.4516, Main Task Loss: 0.4370, RSF Loss: 0.1466
Epoch 110, Total Loss: 0.4482, Main Task Loss: 0.4338, RSF Loss: 0.1438
Epoch 120, Total Loss: 0.4554, Main Task Loss: 0.4409, RSF Loss: 0.1456
Epoch 130, Total Loss: 0.4483, Main Task Loss: 0.4339, RSF Loss:

## Compare Results(GAT)

In [ ]:
import pandas as pd
results = {
    'Model': ['GAT', 'GAT Reweighted', 'GAT Reject Option', 'GAT PRR', 'GAT RSF'],
    'Test Accuracy': [test_acc_gat, test_acc_gat_reweighted, test_acc_gat_reject, test_acc_gat_prr, test_acc_gat_rsf],
    'Rejection Rate': [0, 0, test_reject_mask_gat.sum().item() / test_mask.size(0), 0, 0],
    'Statistical Parity': [fairness_gat.statistical_parity(), fairness_gat_reweighted.statistical_parity(),
                           fairness_gat_reject.statistical_parity(), fairness_gat_prr.statistical_parity(),
                           fairness_gat_rsf.statistical_parity()],
    'Equal Opportunity': [fairness_gat.equal_opportunity(), fairness_gat_reweighted.equal_opportunity(),
                          fairness_gat_reject.equal_opportunity(), fairness_gat_prr.equal_opportunity(),
                          fairness_gat_rsf.equal_opportunity()],
    'Overall Accuracy Equality': [fairness_gat.overall_accuracy_equality(), fairness_gat_reweighted.overall_accuracy_equality(),
                                 fairness_gat_reject.overall_accuracy_equality(), fairness_gat_prr.overall_accuracy_equality(),
                                 fairness_gat_rsf.overall_accuracy_equality()],
    'Treatment Equality': [fairness_gat.treatment_equality(), fairness_gat_reweighted.treatment_equality(),
                           fairness_gat_reject.treatment_equality(), fairness_gat_prr.treatment_equality(),
                           fairness_gat_rsf.treatment_equality()]
}

# Create a Pandas DataFrame from the results dictionary
df_results = pd.DataFrame(results)

# Display the DataFrame as a nicely formatted table
from IPython.display import display
display(df_results)


Statistical Parity Difference (SPD): 0.0044
Statistical Parity Difference (SPD): 0.0017
Statistical Parity Difference (SPD): 0.0024
Statistical Parity Difference (SPD): 0.0065
Statistical Parity Difference (SPD): 0.0066
Equal Opportunity Difference (EOD): 0.0075
Equal Opportunity Difference (EOD): 0.0017
Equal Opportunity Difference (EOD): 0.0106
Equal Opportunity Difference (EOD): 0.0111
Equal Opportunity Difference (EOD): 0.0075
Overall Accuracy Equality Difference (OAED): 0.0036
Overall Accuracy Equality Difference (OAED): 0.0006
Overall Accuracy Equality Difference (OAED): 0.0133
Overall Accuracy Equality Difference (OAED): 0.0056
Overall Accuracy Equality Difference (OAED): 0.0008
Treatment Equality Difference (TED): 0.0077
Treatment Equality Difference (TED): 0.0062
Treatment Equality Difference (TED): 0.0041
Treatment Equality Difference (TED): 0.0102
Treatment Equality Difference (TED): 0.0125


,Model,Test Accuracy,Rejection Rate,Statistical Parity,Equal Opportunity,Overall Accuracy Equality,Treatment Equality
0,GAT,0.828343,0.000000,None,None,None,None
1,GAT Reweighted,0.709504,0.000000,None,None,None,None
2,GAT Reject Option,0.883417,0.176877,None,None,None,None
3,GAT PRR,0.826654,0.000000,None,None,None,None
4,GAT RSF,0.826961,0.000000,None,None,None,None


# GIN Model

In [ ]:
import pandas as pd
from torch_geometric.nn import GINConv, Linear

class GIN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GIN, self).__init__()
        self.conv1 = GINConv(
            torch.nn.Sequential(
                Linear(input_dim, hidden_dim),
                torch.nn.ReLU(),
                Linear(hidden_dim, hidden_dim)
            )
        )
        self.conv2 = GINConv(
            torch.nn.Sequential(
                Linear(hidden_dim, hidden_dim),
                torch.nn.ReLU(),
                Linear(hidden_dim, hidden_dim)
            )
        )
        self.lin = Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        return F.log_softmax(x, dim=1)

# Initialize GIN model
model_gin = GIN(input_dim, hidden_dim, output_dim)

# Define optimizer
optimizer_gin = torch.optim.Adam(model_gin.parameters(), lr=0.01)

# Training function for GIN
def train_gin():
    model_gin.train()
    optimizer_gin.zero_grad()
    out = model_gin(data)[train_mask]
    loss = criterion(out, data.y[train_mask])
    loss.backward()
    optimizer_gin.step()
    return loss.item()

# Train the GIN model
epochs = 200
for epoch in range(epochs):
    train_loss = train_gin()
    if epoch % 10 == 0:
        print(f"Epoch {epoch:03d}, Train Loss (GIN): {train_loss:.4f}")

# Evaluate the GIN model
model_gin.eval()
out = model_gin(data)
logits = out[test_mask]
preds_gin = logits.max(1)[1]
test_acc_gin = preds_gin.eq(data.y[test_mask]).sum().item() / test_mask.size(0)
print(f"Test Accuracy (GIN): {test_acc_gin:.4f}")

# Evaluate fairness for GIN
fairness_gin = Fairness(df_profile, test_mask.numpy(), data.y[test_mask].cpu().numpy(), preds_gin.cpu().numpy(), 'gender')
fairness_gin.statistical_parity()
fairness_gin.equal_opportunity()
fairness_gin.overall_accuracy_equality()
fairness_gin.treatment_equality()

## Reweighting(GIN)


In [ ]:
# --- Reweighting on GIN ---

import numpy as np

# Calculate weights (same as before)
sample_weights = compute_class_weight('balanced', classes=np.unique(data.y[train_mask].cpu().numpy()), y=data.y[train_mask].cpu().numpy())
sample_weights = torch.tensor(sample_weights, dtype=torch.float32)

# Reweighted loss function (same as before)
criterion_reweighted = torch.nn.NLLLoss(weight=sample_weights)

# Initialize a new GIN model for reweighting
model_gin_reweighted = GIN(input_dim, hidden_dim, output_dim)

# Define optimizer
optimizer_gin_reweighted = torch.optim.Adam(model_gin_reweighted.parameters(), lr=0.01)

# Training function with reweighting for GIN
def train_gin_reweighted():
    model_gin_reweighted.train()
    optimizer_gin_reweighted.zero_grad()
    out = model_gin_reweighted(data)[train_mask]
    loss = criterion_reweighted(out, data.y[train_mask])
    loss.backward()
    optimizer_gin_reweighted.step()
    return loss.item()

# Train the reweighted GIN model
epochs = 200
for epoch in range(epochs):
    train_loss = train_gin_reweighted()
    if epoch % 10 == 0:
        print(f"Epoch {epoch:03d}, Train Loss (GIN Reweighted): {train_loss:.4f}")

# Evaluate the reweighted GIN model
model_gin_reweighted.eval()
out = model_gin_reweighted(data)
logits = out[test_mask]
preds_gin_reweighted = logits.max(1)[1]
test_acc_gin_reweighted = preds_gin_reweighted.eq(data.y[test_mask]).sum().item() / test_mask.size(0)
print(f"Test Accuracy (GIN Reweighted): {test_acc_gin_reweighted:.4f}")

# Evaluate fairness for the reweighted GIN model
fairness_gin_reweighted = Fairness(df_profile, test_mask.numpy(), data.y[test_mask].cpu().numpy(), preds_gin_reweighted.cpu().numpy(), 'gender')
fairness_gin_reweighted.statistical_parity()
fairness_gin_reweighted.equal_opportunity()
fairness_gin_reweighted.overall_accuracy_equality()
fairness_gin_reweighted.treatment_equality()


##Reject Option Classifier (GIN)

In [ ]:
# --- Reject Option Classifier on GIN ---

# Initialize a new GIN model for reject option classification
model_gin_reject = GIN(input_dim, hidden_dim, output_dim)

# Define optimizer
optimizer_gin_reject = torch.optim.Adam(model_gin_reject.parameters(), lr=0.01)

# Training function (same as the original training function for GIN)
def train_gin_reject():
    model_gin_reject.train()
    optimizer_gin_reject.zero_grad()
    out = model_gin_reject(data)[train_mask]
    loss = criterion(out, data.y[train_mask])
    loss.backward()
    optimizer_gin_reject.step()
    return loss.item()

# Train the GIN model for reject option classification
epochs = 200
for epoch in range(epochs):
    train_loss = train_gin_reject()
    if epoch % 10 == 0:
        print(f"Epoch {epoch:03d}, Train Loss (GIN Reject Option): {train_loss:.4f}")

# Evaluation function with reject option (same as before)
def evaluate_reject(mask):
    model_gin_reject.eval()
    out = model_gin_reject(data)
    logits = out[mask]
    probs = F.softmax(logits, dim=1)

    # Get predicted classes
    preds = logits.max(1)[1]

    # Identify instances to reject (where the maximum probability is below the threshold)
    reject_mask = probs.max(1)[0] < threshold

    # Calculate accuracy on non-rejected instances
    non_rejected_mask = ~reject_mask
    num_non_rejected = non_rejected_mask.sum().item()
    if num_non_rejected > 0:
        acc = preds[non_rejected_mask].eq(data.y[mask][non_rejected_mask]).sum().item() / num_non_rejected
    else:
        acc = 0  # Handle the case where all instances are rejected

    return acc, preds, reject_mask

# Evaluate the GIN model with reject option
test_acc_gin_reject, test_preds_gin_reject, test_reject_mask_gin = evaluate_reject(test_mask)
print(f"Test Accuracy (GIN Reject Option): {test_acc_gin_reject:.4f}")
print(f"Rejection Rate (GIN Reject Option): {test_reject_mask_gin.sum().item() / test_mask.size(0):.4f}")

# (Only evaluating the fairness on the non-rejected instances)
fairness_gin_reject = Fairness(df_profile, test_mask.numpy()[~test_reject_mask_gin.cpu().numpy()],
                            data.y[test_mask][~test_reject_mask_gin].cpu().numpy(),
                            test_preds_gin_reject[~test_reject_mask_gin].cpu().numpy(), 'gender')
fairness_gin_reject.statistical_parity()
fairness_gin_reject.equal_opportunity()
fairness_gin_reject.overall_accuracy_equality()
fairness_gin_reject.treatment_equality()

## Prejudice Remover Regularizer(GAT)

In [ ]:
# --- Prejudice Remover Regularizer on GAT ---

def calculate_prr_term_gat(model, data, mask, sensitive_attr):
    out = model(data)[mask]
    probs = F.softmax(out, dim=1)

    # Calculate the mean probabilities for each sensitive attribute value
    mean_probs_0 = probs[torch.tensor(df_profile[sensitive_attr].values[mask.numpy()] == 0, dtype=torch.bool)].mean(dim=0)
    mean_probs_1 = probs[torch.tensor(df_profile[sensitive_attr].values[mask.numpy()] == 1, dtype=torch.bool)].mean(dim=0)

    # Calculate the PRR term (squared Euclidean distance between mean probabilities)
    prr_term = torch.sum((mean_probs_0 - mean_probs_1) ** 2)
    return prr_term

# Training function with PRR for GAT
def train_gat_prr(eta=0.1):  # eta controls the strength of the PRR term
    model_gat_prr.train()
    optimizer_gat_prr.zero_grad()

    loss_main = criterion(model_gat_prr(data)[train_mask], data.y[train_mask])

    loss_prr = 0
    for attr in ['gender']:
        loss_prr += calculate_prr_term_gat(model_gat_prr, data, train_mask, attr)

    total_loss = loss_main + eta * loss_prr
    total_loss.backward()
    optimizer_gat_prr.step()
    return total_loss.item(), loss_main.item(), loss_prr.item()

# Initialize a new GAT model for PRR
model_gat_prr = GAT(input_dim, hidden_dim, output_dim)

# Define optimizer
optimizer_gat_prr = torch.optim.Adam(model_gat_prr.parameters(), lr=0.01)

# Train the GAT model with PRR
epochs = 200
for epoch in range(epochs):
    loss, loss_main, loss_prr = train_gat_prr()
    if epoch % 10 == 0:
        print(f"Epoch {epoch:03d}, Total Loss: {loss:.4f}, Main Task Loss: {loss_main:.4f}, PRR Loss: {loss_prr:.4f}")

# Evaluate the GAT model after PRR mitigation
model_gat_prr.eval()
out = model_gat_prr(data)
logits = out[test_mask]
preds_gat_prr = logits.max(1)[1]
test_acc_gat_prr = preds_gat_prr.eq(data.y[test_mask]).sum().item() / test_mask.size(0)
print(f"Test Accuracy (GAT After PRR): {test_acc_gat_prr:.4f}")

# Re-evaluate fairness after PRR mitigation for GAT
fairness_gat_prr = Fairness(df_profile, test_mask.numpy(), data.y[test_mask].cpu().numpy(), preds_gat_prr.cpu().numpy(), 'gender')
fairness_gat_prr.statistical_parity()
fairness_gat_prr.equal_opportunity()
fairness_gat_prr.overall_accuracy_equality()
fairness_gat_prr.treatment_equality()


##Rich Subgroup Fairness (GIN)

In [ ]:
# --- Rich Subgroup Fairness (RSF) on GIN ---

# Calculate the RSF term for GIN
def calculate_rsf_term_gin(model, data, mask, sensitive_attr, gamma=0.1):
    out = model(data)[mask]
    probs = F.softmax(out, dim=1)

    # Calculate the empirical risk for each subgroup
    risk_0 = F.nll_loss(probs[torch.tensor(df_profile[sensitive_attr].values[mask.numpy()] == 0, dtype=torch.bool)],
                       data.y[mask][torch.tensor(df_profile[sensitive_attr].values[mask.numpy()] == 0, dtype=torch.bool)])
    risk_1 = F.nll_loss(probs[torch.tensor(df_profile[sensitive_attr].values[mask.numpy()] == 1, dtype=torch.bool)],
                       data.y[mask][torch.tensor(df_profile[sensitive_attr].values[mask.numpy()] == 1, dtype=torch.bool)])

    # Calculate the RSF term
    rsf_term = torch.abs(risk_0 - risk_1) - gamma * (risk_0 + risk_1)
    return rsf_term

# Initialize a new GIN model for RSF
model_gin_rsf = GIN(input_dim, hidden_dim, output_dim)

# Define optimizer
optimizer_gin_rsf = torch.optim.Adam(model_gin_rsf.parameters(), lr=0.01)

# Training function with RSF for GIN
def train_gin_rsf(lambda_=0.1):  # lambda_ controls the strength of the RSF term
    model_gin_rsf.train()
    optimizer_gin_rsf.zero_grad()

    loss_main = criterion(model_gin_rsf(data)[train_mask], data.y[train_mask])

    loss_rsf = calculate_rsf_term_gin(model_gin_rsf, data, train_mask, sensitive_attr)

    total_loss = loss_main + lambda_ * loss_rsf
    total_loss.backward()
    optimizer_gin_rsf.step()
    return total_loss.item(), loss_main.item(), loss_rsf.item()

# Train the GIN model with RSF
epochs = 200
for epoch in range(epochs):
    loss, loss_main, loss_rsf = train_gin_rsf()
    if epoch % 10 == 0:
        print(f"Epoch {epoch:03d}, Total Loss: {loss:.4f}, Main Task Loss: {loss_main:.4f}, RSF Loss: {loss_rsf:.4f}")

# Evaluate the GIN model after RSF mitigation
model_gin_rsf.eval()
out = model_gin_rsf(data)
logits = out[test_mask]
preds_gin_rsf = logits.max(1)[1]
test_acc_gin_rsf = preds_gin_rsf.eq(data.y[test_mask]).sum().item() / test_mask.size(0)
print(f"Test Accuracy (GIN After RSF): {test_acc_gin_rsf:.4f}")

# Re-evaluate fairness after RSF mitigation for GIN
fairness_gin_rsf = Fairness(df_profile, test_mask.numpy(), data.y[test_mask].cpu().numpy(), preds_gin_rsf.cpu().numpy(), 'gender')
fairness_gin_rsf.statistical_parity()
fairness_gin_rsf.equal_opportunity()
fairness_gin_rsf.overall_accuracy_equality()
fairness_gin_rsf.treatment_equality()